In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate
!pip install streamlit -q
!pip install accelerate
!pip install bitsandbytes


  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/25740/head) to /tmp/pip-req-build-f8ob5gi3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-f8ob5gi3
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/25740/head
  Running command git checkout -q c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Resolved https://github.com/huggingface/transformers.git to commit c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
%%writefile CodeModel.py

# CodeModel.py

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
import streamlit as st
import re

@st.cache_resource
def load_model():
    model_id = "codellama/CodeLlama-7b-Instruct-hf"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    bnb_config = BitsAndBytesConfig(load_in_8bit=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.float16
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )
    return pipe, tokenizer

pipe, tokenizer = load_model()



def generate_code(user):

    def truncate_to_last_sentence(text):
        # Remove hanging bullet like "19." at end
        match = re.search(r"(.*?)(\n\d+\.\s*)?$", text.strip(), re.DOTALL)
        if match:
            text = match.group(1)

        # Now trim to last full sentence
        if '.' in text:
            last_period = text.rfind('.')
            return text[:last_period + 1].strip()
        return text.strip()


    # system = "You are CodeGenie, a concise programming tutor. Summarize what the code does, evaluate logic and style, then suggest clear fixes."
    system = "You are CodeGenie, a concise programming tutor. Summarize what the code does, evaluate logic and style, then suggest clear fixes."
    '''user = """Analyze this Python function and give feedback:

    def add(a, b):
      return a + b"""'''
    prompt = f"<s><<SYS>>\n{system}\n<</SYS>>\n\n{user}"

    prompt_tokens = tokenizer(prompt, return_tensors="pt").input_ids.shape[-1]
    max_total_tokens = 256
    max_new_tokens = max(32, max_total_tokens - prompt_tokens)

    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=max_new_tokens,
        add_special_tokens=False
    )

    generated = sequences[0]['generated_text']
    if prompt in generated:
        result = generated.replace(prompt, "").strip()
    else:
        # In case model doesn't echo prompt, just return from the first new line
        result = generated.split('\n', 1)[-1].strip()

    result = truncate_to_last_sentence(result)

    return result

Overwriting CodeModel.py


In [ ]:
%%writefile app.py

import streamlit as st
import time
from CodeModel import generate_code
st.header("CodeGenie: AI-powered code assistant")

with st.form("my_form"):
    user_input = st.text_area("Enter your text prompt below and click the button to submit.")
    submit = st.form_submit_button(label="Submit text prompt")

if submit:
    with st.spinner(text="Generating code... It may take some time"):
        code = generate_code(user=user_input)
        print(code)
        st.code(code, language='python')
    st.sidebar.markdown("## Guide")
    st.sidebar.info("This tool uses CodeLlama 7B parameters")

Overwriting app.py


In [7]:
!wget -q -O - ipv4.icanhazip.com

35.233.133.239


**Copy that number and paste it on the website as password on next cell**

**Click "your url is:" link and paste that number**


In [12]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.2:8501
  External URL: http://35.233.133.239:8501

⠼⠴your url is: https://some-grapes-pay.loca.lt
2025-06-18 08:19:28.022423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750234768.047023   17278 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750234768.054115   17278 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch


In [ ]:

model = "codellama/CodeLlama-7b-Instruct-hf" #"codellama/CodeLlama-7b-hf"
messages=[
        {"role": "system", "content": "You are a helpful AI bot. Your name is Bob."},
        {"role": "user", "content": "Hello, how are you doing?"},
        {"role": "assistant", "content": "I'm doing well, thanks!"},
        {"role": "user", "content": "What is your name?"},
    ],



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# system = "Provide insights into code efficiency, syntax correctness, and logical flow. Also offer suggestions for improvement and highlight specific parts of the code that are particularly well-written or problematic."
system = "You are CodeGenie, a concise programming tutor. Summarize what the code does, evaluate logic and style, then suggest clear fixes.""
user = "Analyze this Python function and give feedback:\n\ndef add(a, b):\nreturn a + b"

prompt = f"<s><<SYS>>\n{system}\n<</SYS>>\n\n{user}"


In [ ]:
prompt_tokens = tokenizer(prompt, return_tensors="pt").input_ids.shape[-1]
max_total = 512
max_new_tokens = max_total - prompt_tokens


In [ ]:
max_new_tokens

421

In [ ]:
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=512,
    add_special_tokens=False
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: <s><<SYS>>
Provide insights into code efficiency, syntax correctness, and logical flow. Also offer suggestions for improvement and highlight specific parts of the code that are particularly well-written or problematic. Also, answer consisely within 200 tokens.
<</SYS>>

Analyze this Python function and give feedback:

def add(a, b):
return a + b

<</ANS>>

This function is a simple addition function. It takes two arguments, a and b, and returns their sum. The function is well-written and easy to understand. However, it could be improved by using a more descriptive name for the function, such as "add_numbers". Additionally, the function could be made more efficient by using the built-in sum() function, which would allow the function to be written in a single line of code.

Here is an improved version of the function:

def add_numbers(a, b):
return sum(a, b)

This function is more efficient and easier to read. It uses the built-in sum() function to perform the addition, which eli